In [1]:
import os
import json
import math
import numpy as np
import pandas as pd
pd.options.display.max_columns = 100
import sys
import pickle
import io
from tqdm import tqdm

from  statsmodels.stats.proportion import proportion_confint
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import precision_recall_fscore_support, classification_report
from tqdm import tqdm

#path_cc - путь к конфигам
#path_union - путь к файлу с текстами для прогона
path_union = '/Users/alexandergrigoriev/work/sandbox/sb900-static/dialogs'
log_path = '/Users/alexandergrigoriev/work/sandbox/sb900-test/log'

answers_path = '/Users/alexandergrigoriev/work/sandbox/sb900-static/references'
pd.set_option('display.max_colwidth', -1)

input_dir = 'input'
test_fname = os.path.join(input_dir, 'test_rnd_lift_100.xlsx')

In [2]:
?tqdm

In [2]:
project_dir = '/Users/alexandergrigoriev/work/sandbox/sb900-ai/'
static_dir = '/Users/alexandergrigoriev/work/sandbox/sb900-static/'
STATIC_PATH = static_dir
os.environ['STATIC_PATH'] = static_dir
sys.path.append(project_dir)
from ai.ai_model.base_text_normalizer import text_normalizer
from ai.ai_model.faq.faq_responder import FaqResponder
from ai.configs.faq_config import FaqConfig

from ai.configs.intent_recognizer_config import IntentRecognizerConfig
from ai.vectorizer.simple_word2vec import SimpleW2V

TextNormalizer init time = 6s


In [3]:
python_path_list = ['/Users/alexandergrigoriev/work/sandbox/sb900-ai']
#                '/Users/alexandergrigoriev/work/sandbox',
#                '/Users/alexandergrigoriev/work/sandbox/sb900-test']
PYTHON_PATH = ':'.join(python_path_list)
os.environ['PYTHONPATH'] = PYTHON_PATH

In [4]:
source = pd.read_excel(test_fname)
source.head()

,mq_id,master_question,phrase,source
2965,9999,Прочее,у меня в сбербанк-он лайн заведена заявка на кредит. сейчас необходимо привезти документы в отделение. но отделение где я заводил заявку не работает. в какое отделение банка в нижнем новгороде я могу сегодня отдать документы?,rnd
17393,9999,Прочее,"здравствуйте, хотела оставить заявку на потребительский кредит.",rnd
4492,9999,Прочее,доброго времени суток. я покупал в интернете товар. оплата была с пополнением яндекс.кошелька. я оплачивал со своей карты visa.,rnd
6566,9999,Прочее,хочу подать заявление на снижение ставки по ипотеке,rnd
124399,1013,Как изменить номер телефона,хотел бы привязать новый номер телефона к карте,nst


In [5]:
source.rename(columns = {'phrase' : 'original_text'}, inplace = True)
source.dropna(subset = ['original_text'], inplace = True)
source.shape

(21113, 4)

In [6]:
def clean_text(text):
    if not '{бот}' in text:
        return text.split('[Клиент')[0]
    else:
        return

source['CLEANTEXT'] = source['original_text'].apply(lambda x: clean_text(x))
source = source.dropna(subset = ['CLEANTEXT'])
source.dropna(subset = ['CLEANTEXT'])
source.shape

(21113, 5)

In [7]:
#немного нормализации
import string
#exclude = string.punctuation
exclude = '\r\n'
import re
def lowerize(text):
    text = ''.join(l  if l not in exclude else ' ' for l in text)
    text = text.lower().strip()
    text = re.sub('[ ]+', ' ', text)
    text = text.replace('и\
̆', 'й')
    return text

source['CLEANTEXT'] = source['CLEANTEXT'].apply(lambda x: lowerize(x))

In [8]:
#заливаем файл для пакетного прогона

dialogs = source[['CLEANTEXT']].copy()
dialogs.rename(columns={'CLEANTEXT': 'message'}, inplace=True)

dialogs['dialog_id'] = source.index  # source.ROW_ID
dialogs['user_id'] = source.index
dialogs['message_id'] = source.index
dialogs['channel'] = 'SBERBANK_MESSENGER'
dialogs['expected_scenario'] = 'Successfully choose base FAQ'
dialogs['expected_master_question'] = ''
dialogs['expected_node_id'] = ''
dialogs['type'] = 'faq'

for index, row in dialogs.iterrows():
    if '\n' in str(row['message']):
        dialogs.loc[index, 'message'] = str(str(row['message']).replace('\n', ' '))

dialogs_columns = ['dialog_id', 'message_id', 'user_id', 'channel', 'message', 'expected_scenario',\
                     'expected_node_id', 'expected_master_question', 'type']

dialogs[dialogs_columns].to_csv(os.path.join(path_union, 'dialogs.csv'), index=False, sep=';', encoding = 'utf-8')

In [9]:
dialogs.shape

(21113, 9)

In [10]:
%%bash
python /Users/alexandergrigoriev/work/sandbox/sb900-ai/tests/system_testing/batch_testing/test_batch_sender.py

TextNormalizer init time = 5s
Used FileLogger
Log destination: /Users/alexandergrigoriev/work/sandbox/sb900-test/log/dialog_23082018_7598.log
Loading dialogs
The dialogues have been loaded
Initializing the AIModel...
The AIModel has been initialized
Running the dialogues...
Parsing the log...
The log has been parsed
Analyzing tests results...
0 tests out of 20066 (0.00%) passed
Scenario guesses: 0 out of 21113 (0.00%)
Node guesses: 21113 out of 21113 (100.00%)
MasterQuestion guesses: 15296 out of 21113 (72.45%)
The tests results have been analyzed
Saving the parsed log...
Saving the test analysis...
UdPipe load: name match ok:morphodita_parsito


21113it [02:34, 136.38it/s]


In [11]:
#читаем после прогона лог
logs = pd.read_csv(os.path.join(log_path,'parsed_log.csv'))
# logs = pd.read_csv('input/parsed_log.csv')
logs.shape

(21113, 19)

In [12]:
logs = logs.rename(columns = {'master_question' : 'mq_id'})

with open (os.path.join(answers_path,'faq_answers.json'), 'r') as f:
    data = json.load(f)
data_dict = {key : data[key]["master_question_action"]["nodes"]["answer"] for key in data.keys() if key != "action"}

logs['mq_id'].fillna(2002, inplace = True)

def parse(t):
    if t != 2002:
        return data_dict[str(int(t))]
    else:
        return "OTHER"
logs['master_question'] = logs['mq_id'].apply(lambda x: parse(x) )

In [13]:
source['mq_id'].replace(9999, 2002, inplace=True)
logs['mq_id'].replace(9999, 2002, inplace=True)

In [14]:
logs['original_text'] = logs['request'].apply(lambda x: json.loads(x)["payload"]["message"])


In [15]:
source['dialog_id'] = source.index
source.rename(columns = {'ROW_ID' : 'dialog_id', 'mq_id' : 'real_mq_id'}, inplace = True)
source.head()

,real_mq_id,master_question,original_text,source,CLEANTEXT,dialog_id
2965,2002,Прочее,у меня в сбербанк-он лайн заведена заявка на кредит. сейчас необходимо привезти документы в отделение. но отделение где я заводил заявку не работает. в какое отделение банка в нижнем новгороде я могу сегодня отдать документы?,rnd,у меня в сбербанк-он лайн заведена заявка на кредит. сейчас необходимо привезти документы в отделение. но отделение где я заводил заявку не работает. в какое отделение банка в нижнем новгороде я могу сегодня отдать документы?,2965
17393,2002,Прочее,"здравствуйте, хотела оставить заявку на потребительский кредит.",rnd,"здравствуйте, хотела оставить заявку на потребительский кредит.",17393
4492,2002,Прочее,доброго времени суток. я покупал в интернете товар. оплата была с пополнением яндекс.кошелька. я оплачивал со своей карты visa.,rnd,доброго времени суток. я покупал в интернете товар. оплата была с пополнением яндекс.кошелька. я оплачивал со своей карты visa.,4492
6566,2002,Прочее,хочу подать заявление на снижение ставки по ипотеке,rnd,хочу подать заявление на снижение ставки по ипотеке,6566
124399,1013,Как изменить номер телефона,хотел бы привязать новый номер телефона к карте,nst,хотел бы привязать новый номер телефона к карте,124399


In [16]:
logs.head(1).T

,0
uid,2965
original_text,у меня в сбербанк-он лайн заведена заявка на кредит. сейчас необходимо привезти документы в отделение. но отделение где я заводил заявку не работает. в какое отделение банка в нижнем новгороде я могу сегодня отдать документы?
message_id,2965
chosen_scenario,can_not_answer
chosen_sub_scenario,NaN
weight,NaN
chosen_node_id,NaN
distance,NaN
mq_id,2002
request,"{""messageId"": ""2965"", ""messageName"": ""MESSAGE_FROM_USER"", ""payload"": {""message"": ""у меня в сбербанк-он лайн заведена заявка на кредит. сейчас необходимо привезти документы в отделение. но отделение где я заводил заявку не работает. в какое отделение банка в нижнем новгороде я могу сегодня отдать документы?"", ""device"": {""client_type"": ""RETAIL"", ""channel"": ""MP_SBOL_IOS"", ""channel_version"": ""8.10.0.233"", ""platform_name"": ""iPhone"", ""platform_version"": ""11.1""}, ""csa_profile_id"": 868}, ""uuid"": {""chatId"": ""2965"", ""userChannel"": ""SBERBANK_MESSENGER"", ""userId"": ""2965""}}"


In [17]:
logs_mq = pd.merge(logs[['master_question', 'mq_id', 'original_text', 'distance', 'dialog_id']],
                   source[['CLEANTEXT',  'real_mq_id', 'dialog_id']], how = 'left',
                   left_on = ['original_text', 'dialog_id'],
                   right_on = ['CLEANTEXT', 'dialog_id'])

logs_mq.head()

,master_question,mq_id,original_text,distance,dialog_id,CLEANTEXT,real_mq_id
0,OTHER,2002.0,у меня в сбербанк-он лайн заведена заявка на кредит. сейчас необходимо привезти документы в отделение. но отделение где я заводил заявку не работает. в какое отделение банка в нижнем новгороде я могу сегодня отдать документы?,NaN,2965,у меня в сбербанк-он лайн заведена заявка на кредит. сейчас необходимо привезти документы в отделение. но отделение где я заводил заявку не работает. в какое отделение банка в нижнем новгороде я могу сегодня отдать документы?,2002
1,OTHER,2002.0,"здравствуйте, хотела оставить заявку на потребительский кредит.",NaN,17393,"здравствуйте, хотела оставить заявку на потребительский кредит.",2002
2,OTHER,2002.0,доброго времени суток. я покупал в интернете товар. оплата была с пополнением яндекс.кошелька. я оплачивал со своей карты visa.,NaN,4492,доброго времени суток. я покупал в интернете товар. оплата была с пополнением яндекс.кошелька. я оплачивал со своей карты visa.,2002
3,Прочее,2002.0,хочу подать заявление на снижение ставки по ипотеке,0.252923,6566,хочу подать заявление на снижение ставки по ипотеке,2002
4,К какому оператору сотовой связи можно подключить МБ,1045.0,хотел бы привязать новый номер телефона к карте,0.241024,124399,хотел бы привязать новый номер телефона к карте,1013


In [18]:
logs_mq[logs_mq['dialog_id']==2341]

,master_question,mq_id,original_text,distance,dialog_id,CLEANTEXT,real_mq_id
534,OTHER,2002.0,"здравствуйте, планирую путешествие в турцию беременность 25-26 недель. есть ли страхование для беременных и какая сумма покрытия",NaN,2341,"здравствуйте, планирую путешествие в турцию беременность 25-26 недель. есть ли страхование для беременных и какая сумма покрытия",2002


In [19]:
source[source['dialog_id']==2341]

,real_mq_id,master_question,original_text,source,CLEANTEXT,dialog_id
2341,2002,Прочее,"здравствуйте, планирую путешествие в турцию беременность 25-26 недель. есть ли страхование для беременных и какая сумма покрытия",rnd,"здравствуйте, планирую путешествие в турцию беременность 25-26 недель. есть ли страхование для беременных и какая сумма покрытия",2341


In [20]:
logs[logs['dialog_id']==2341].T

,534
uid,2341
original_text,"здравствуйте, планирую путешествие в турцию беременность 25-26 недель. есть ли страхование для беременных и какая сумма покрытия"
message_id,2341
chosen_scenario,can_not_answer
chosen_sub_scenario,NaN
weight,NaN
chosen_node_id,NaN
distance,NaN
mq_id,2002
request,"{""messageId"": ""2341"", ""messageName"": ""MESSAGE_FROM_USER"", ""payload"": {""message"": ""здравствуйте, планирую путешествие в турцию беременность 25-26 недель. есть ли страхование для беременных и какая сумма покрытия"", ""device"": {""client_type"": ""RETAIL"", ""channel"": ""MP_SBOL_IOS"", ""channel_version"": ""8.10.0.233"", ""platform_name"": ""iPhone"", ""platform_version"": ""11.1""}, ""csa_profile_id"": 868}, ""uuid"": {""chatId"": ""2341"", ""userChannel"": ""SBERBANK_MESSENGER"", ""userId"": ""2341""}}"


In [21]:
logs_mq = logs_mq.dropna(subset = ['original_text'])
logs.shape

(21113, 20)

In [22]:
def real(mq):
    if mq == 'Прочее':
        return 'OTHER'
    else:
        return mq

In [23]:
# logs_mq['real_master_question'] = logs_mq['real_master_question'].apply(lambda x: real(x)) 
logs_mq['real_mq_id'].fillna(2002, inplace = True)
#logs_mq.to_csv('logs_mq.csv', sep ='\t')

In [24]:
def other(row):
    #print (row['mq_id'], row['real_mq_id'])
    if row['mq_id'] == row['real_mq_id']:
        row['tex_real_mq_id'] = row['mq_id']
       
    else:
        row['tex_real_mq_id'] = 2002

    return row

    
def mask(ID):
    if ID == 2002:
        return 0
    else:
        return 1    

In [25]:
logs_mq.to_excel('test_logs_validated.xlsx')
#logs_mq = pd.read_excel('38_faq/test_logs_baseline.xlsx')

In [26]:
logs_mq.shape

(21113, 7)

In [27]:
import math
def make_results(source):
    false_positives, Prs, mqs, mq_ids, cis, lows, ups, fps, tps = [], [], [], [], [],[],[], [],[]
    false_negatives, fns = [], []
    #for mq_id in tqdm(mq_id_list):
    id_lst = source.mq_id.unique()
    for mq_id in tqdm(id_lst):

            tmp = source[source['mq_id'] == mq_id]

            #if tmp.shape[0] > 0:
            mq = tmp[tmp['mq_id'] == mq_id].iloc[0]['master_question']  
            mqs.append(mq)

            tmp = tmp.apply(lambda x: other(x), axis = 1)
            tmp['tex_real_mq_id'] = tmp.tex_real_mq_id.apply(lambda x: mask(x))

            tmp['tex_mq_id'] = tmp['mq_id'].apply(lambda x: mask(x))     
            f_p = tmp[(tmp.tex_mq_id == 1) & (tmp.tex_real_mq_id == 0)]\
            [['dialog_id', 'original_text', 'master_question', 'mq_id', 'real_mq_id', 'distance']]
            
            f_n = source[(source.real_mq_id == mq_id) & (source.mq_id != mq_id)]\
            [['dialog_id', 'original_text', 'master_question', 'mq_id', 'real_mq_id', 'distance']]

            false_positives.append(f_p)
            false_negatives.append(f_n)
            precision = precision_score(tmp.tex_real_mq_id.values, tmp.tex_mq_id.values)
            Prs.append(precision)
            fp, tp = confusion_matrix(tmp.tex_real_mq_id.values, tmp.tex_mq_id.values, labels = [0,1])[:,1]
            
            ci = proportion_confint(tp, tp + fp, method = 'wilson')
            cis.append(ci)
            lows.append(ci[0])
            ups.append(ci[1])
            fps.append(fp)
            fns.append(f_n.shape[0])
            tps.append(tp)
            

    false_positives = pd.concat(false_positives)
    false_negatives = pd.concat(false_negatives)
    #true_positives = pd.concat(true_positives)

    result = pd.DataFrame({'mq_id' : id_lst, 'precision' : Prs, 'mq' : mqs,\
                           'precision_lb' : lows, 'precision_ub' : ups, 
                          'tp' : tps, 'fp' : fps, 'fn' : fns})
    result = result[['mq', 'mq_id', 'precision', 'precision_lb', 'precision_ub', 'tp', 'fp', 'fn']]
    return result, false_positives, false_negatives

In [28]:
result, false_positives, false_negatives = make_results(logs_mq)

  0%|          | 0/41 [00:00<?, ?it/s]/anaconda3/envs/sb900_alex/lib/python3.6/site-packages/statsmodels/stats/proportion.py:85: RuntimeWarning: invalid value encountered in double_scalars
  q_ = count * 1. / nobs
/anaconda3/envs/sb900_alex/lib/python3.6/site-packages/statsmodels/stats/proportion.py:130: RuntimeWarning: divide by zero encountered in double_scalars
  denom = 1 + crit2 / nobs
/anaconda3/envs/sb900_alex/lib/python3.6/site-packages/statsmodels/stats/proportion.py:131: RuntimeWarning: divide by zero encountered in double_scalars
  center = (q_ + crit2 / (2 * nobs)) / denom
/anaconda3/envs/sb900_alex/lib/python3.6/site-packages/statsmodels/stats/proportion.py:132: RuntimeWarning: divide by zero encountered in double_scalars
  dist = crit * np.sqrt(q_ * (1. - q_) / nobs + crit2 / (4. * nobs**2))
100%|██████████| 41/41 [00:11<00:00,  3.63it/s]


In [29]:
result.sort_values(by = 'precision_lb', ascending = False)

,mq,mq_id,precision,precision_lb,precision_ub,tp,fp,fn
2,Здравствуйте,1011.0,0.973114,0.963004,0.980517,1303,36,263
22,Какой режим работы банка,1019.0,0.956731,0.919826,0.977072,199,9,197
4,До свидания,1006.0,1.000000,0.910333,1.000000,39,0,46
25,Как подключить опцию быстрый платеж в МБ,1032.0,0.978723,0.888870,0.996234,46,1,31
30,Как отключить услугу бесконтактная оплата,1048.0,0.957447,0.857515,0.988252,45,2,38
31,Как получить договор купли-продажи,1007.0,0.971429,0.854669,0.994939,34,1,45
21,Как совершить перевод с qiwi,1028.0,0.967742,0.838059,0.994283,30,1,49
28,Как купить акции,1018.0,0.944444,0.818553,0.984630,34,2,52
35,Как устроиться в Сбербанк,1044.0,0.960000,0.804559,0.992904,24,1,52
9,Как отключить копилку,1023.0,0.860000,0.738138,0.930492,43,7,42


In [30]:
result.sort_values(by = 'precision_lb', ascending = False).to_excel('precision.xlsx')

In [31]:
false_positives[false_positives.mq_id == 1058]

,dialog_id,original_text,master_question,mq_id,real_mq_id,distance


In [32]:
recalls = pd.DataFrame({'mq_id' : logs_mq['mq_id'].unique(),\
                        'recall' : recall_score(logs_mq['real_mq_id'].values,\
                                                logs_mq['mq_id'].values, labels = logs_mq['mq_id'].unique(), average = None)})

In [33]:
#result = result[result.mq_id.isin(mq_id_list)]
recalls.shape,result.shape

((41, 2), (41, 8))

In [34]:
result = result.merge(recalls, on = ['mq_id'])

In [35]:
false_positives.to_excel('fp_fn/fp_.xlsx')
false_negatives.to_excel('fp_fn/fn_.xlsx')

In [36]:
logs_mq.head()

,master_question,mq_id,original_text,distance,dialog_id,CLEANTEXT,real_mq_id
0,OTHER,2002.0,у меня в сбербанк-он лайн заведена заявка на кредит. сейчас необходимо привезти документы в отделение. но отделение где я заводил заявку не работает. в какое отделение банка в нижнем новгороде я могу сегодня отдать документы?,NaN,2965,у меня в сбербанк-он лайн заведена заявка на кредит. сейчас необходимо привезти документы в отделение. но отделение где я заводил заявку не работает. в какое отделение банка в нижнем новгороде я могу сегодня отдать документы?,2002
1,OTHER,2002.0,"здравствуйте, хотела оставить заявку на потребительский кредит.",NaN,17393,"здравствуйте, хотела оставить заявку на потребительский кредит.",2002
2,OTHER,2002.0,доброго времени суток. я покупал в интернете товар. оплата была с пополнением яндекс.кошелька. я оплачивал со своей карты visa.,NaN,4492,доброго времени суток. я покупал в интернете товар. оплата была с пополнением яндекс.кошелька. я оплачивал со своей карты visa.,2002
3,Прочее,2002.0,хочу подать заявление на снижение ставки по ипотеке,0.252923,6566,хочу подать заявление на снижение ставки по ипотеке,2002
4,К какому оператору сотовой связи можно подключить МБ,1045.0,хотел бы привязать новый номер телефона к карте,0.241024,124399,хотел бы привязать новый номер телефона к карте,1013


In [37]:
logs_mq['mq_id'] = logs_mq['mq_id'].astype(int)

In [38]:
logs_mq['mq_id'].replace({2002: 9999}, inplace=True)
logs_mq['real_mq_id'].replace({2002: 9999}, inplace=True)

In [39]:
y_true = logs_mq['real_mq_id']
y_pred = logs_mq['mq_id']

## F-score

In [40]:
from sklearn.metrics import precision_recall_fscore_support, fbeta_score

def get_f_scores(y_true, y_pred):
    labels = sorted(np.unique(y_true))
#     print(labels)
    _, _, f05score, support = precision_recall_fscore_support(y_true, y_pred,average=None, beta=0.5,labels=labels)
    classes_score = (f05score[:-1] * support[:-1]).sum()/support[:-1].sum()
    f2_scores = fbeta_score(y_true, y_pred, average=None, beta=2,labels=labels)
    other_score = f2_scores[-1]
#     print(f2_scores)
    result_score = (classes_score + other_score) / 2
    
    return {"classes_score": classes_score, 
            "other_score": other_score, 
            "result_score": result_score}

In [41]:
f_scores = get_f_scores(y_true, y_pred)
f_scores

{'classes_score': 0.6737999926901127,
 'other_score': 0.9540257993519593,
 'result_score': 0.813912896021036}